In [5]:
# general
import numpy as np
import pandas as pd
from IPython.display import display
import importlib

# first used in exercise one
import kernelsvm as svm
from sklearn.metrics.pairwise import linear_kernel, polynomial_kernel
from sklearn.datasets import fetch_mldata
from sklearn import preprocessing
from sklearn.model_selection import train_test_split

In [6]:
importlib.reload(svm)

<module 'kernelsvm' from '/Users/andrewenfield/work/github/Data558/Week09/kernelsvm.py'>

In [7]:
x_simple = np.array([3,2,0,1,-1,-2]).reshape(3,2)
x_simple

array([[ 3,  2],
       [ 0,  1],
       [-1, -2]])

Note: Per the request in the "Collaboration policy" note, I've discussed at least part of this assignment with many of the MS employees in the class, including Abhishek, Geoff, Suman, and Charles. (Different weeks/different assignments have different people, depending upon who attends our study groups, but I'll probably just include this blurb w/ each homework since it's generally correct.) I've also gotten input from the discussion board.

# Exercise one

_Compute the gradient ∇F(α) of F._

![FirstProb](FirstProb.jpg)

_Write a function computegram that computes, for any set of datapoints x1, . . . , xn, the kernel matrix K._

I implemented the computegram_linear function, and all of the supporting functions including the gradient and objective functions, in the file kernelsvm.py, which I imported into this notebook with the alias svm. I also include my (relatively minimal) unit tests, in kernelsvm-test.py.

In [8]:
svm.computegram_linear(x_simple)

array([[13,  2, -7],
       [ 2,  1, -2],
       [-7, -2,  5]])

In [9]:
svm.computegram_polynomial(x_simple, 2, 1)

array([[196,   9,  36],
       [  9,   4,   1],
       [ 36,   1,  36]])

_Write a function kerneleval that computes, for any set of datapoints x1, . . . , xn and a new datapoint x⋆, the vector of kernel evaluations [k(x1, x⋆), . . . , k(xn, x⋆)]T._

I implemented this function in kernelsvm.py.

My understanding here is that I'm not using the kernel matrix - that is, the reference to 'kernel evaluation' means to use the kernel _function_, not the kernel matrix. I think this is just (part of - since no alpha weighting yet) the predict step, which - for a linear kernel - is the sum of the inner products between each observation and the value to be predicted, weighted by each learned value of alpha.

In [10]:
svm.kerneleval_linear(x_simple, np.array([2,2]))

array([10,  2, -6])

_Consider the MNIST dataset. You can find instructions on how to download it here: http://scikit-learn.org/stable/datasets/mldata.html. Pick two classes of your choice. Your are going to work on the dataset consisting of the data from these two classes. Standardize the data, if you have not done so already._

In [11]:
mnist = fetch_mldata('MNIST original')
mnist.data.shape, mnist.target.shape

((70000, 784), (70000,))

We'll use 1 and 8. 

In [12]:
ones_and_eights = (mnist.target == 1) | (mnist.target == 8)
X = mnist.data[ones_and_eights]
y = mnist.target[ones_and_eights]
X.shape, y.shape, np.unique(y)

((14702, 784), (14702,), array([ 1.,  8.]))

In [13]:
X_scaled = preprocessing.scale(X)
X_scaled.shape

/Users/andrewenfield/miniconda3/envs/data558/lib/python3.6/site-packages/sklearn/utils/validation.py:429: DataConversionWarning: Data with input dtype uint8 was converted to float64 by the scale function.
  warnings.warn(msg, _DataConversionWarning)


(14702, 784)

In [14]:
X_scaled_train, X_scaled_test, y_train, y_test = train_test_split(X_scaled, y)
X_scaled_train.shape, X_scaled_test.shape, y_train.shape, y_test.shape

((11026, 784), (3676, 784), (11026,), (3676,))

_Write a function mysvm that implements the fast gradient algorithm to train the kernel support vector machine with the squared hinge loss. The function takes as input the initial step-size value for the backtracking rule and a maximum number of iterations._

As noted above, I implemented this in the kernelsvm.py file. Instead of creating a function named mysvm, I just call my fastgradalgo function and pass in references to the kernel SVM gradient and objective functions (which are also implemented in the same file).

_Train your kernel support vector machine with the squared hinge loss and the polyno- mial kernel of order 7 on the the MNIST dataset, tuning the regularization parameter λ using cross-validation._

I tried both my fast gradient descent and gradient descent implementations with the kernel SVM gradient and objective functions, but haven't been able to get it to work, from what I can tell. Running for even just 10 iterations takes much longer than it should, to start with - this shouldn't be the case, I think (?): it should run more quickly?. And then the coefficients that I get make me think the algorithm hasn't converged - sometimes they're massively large and other times very small (effectively zero).

While it took until the end of the last part of the last exercise of the last homework, I'm going to throw in the towel on this one. I still have work to do to finish the final project report, and don't have more time to spend on this assignment.

In [15]:
importlib.reload(svm)

<module 'kernelsvm' from '/Users/andrewenfield/work/github/Data558/Week09/kernelsvm.py'>

In [16]:
t_init = 0.01
max_iters = 10

In [24]:
results = svm.fastgradalgo(
    svm.computegram_linear(X_scaled_train), 
    y_train, t_init=t_init, 
    grad_func = svm.compute_kernelsvm_gradient, 
    obj_func = svm.compute_kernelsvm_objective, 
    lam=1, max_iter=max_iters, t_func=svm.backtracking)
results[-3:]

,0,1,2,3,4,5,6,7,8,9,...,11016,11017,11018,11019,11020,11021,11022,11023,11024,11025
8,-5.462164e-08,-1.401092e-07,-5.213958e-08,1.071975e-07,-1.389692e-08,-3.648095e-08,0.000001,-6.721888e-08,-5.752141e-08,-8.009989e-08,...,-6.142063e-08,7.656160e-08,8.296727e-07,-4.297519e-08,8.620134e-07,2.374047e-08,4.307277e-07,-5.100345e-07,-5.320968e-08,1.450999e-08
9,-5.462164e-08,-1.401092e-07,-5.213958e-08,1.071975e-07,-1.389692e-08,-3.648095e-08,0.000001,-6.721888e-08,-5.752141e-08,-8.009989e-08,...,-6.142063e-08,7.656160e-08,8.296727e-07,-4.297519e-08,8.620134e-07,2.374047e-08,4.307277e-07,-5.100345e-07,-5.320968e-08,1.450999e-08
10,-5.462164e-08,-1.401092e-07,-5.213958e-08,1.071975e-07,-1.389692e-08,-3.648095e-08,0.000001,-6.721888e-08,-5.752141e-08,-8.009989e-08,...,-6.142063e-08,7.656160e-08,8.296727e-07,-4.297519e-08,8.620134e-07,2.374047e-08,4.307277e-07,-5.100345e-07,-5.320968e-08,1.450999e-08


In [27]:
results_regular = svm.graddescent(np.zeros(len(X_scaled_train)), 
                svm.computegram_linear(X_scaled_train), y_train,
                t_init=t_init,
                grad_func = svm.compute_kernelsvm_gradient, 
                obj_func = svm.compute_kernelsvm_objective, 
                lam=1, max_iter=max_iters)
results_regular[-3:]

,0,1,2,3,4,5,6,7,8,9,...,11016,11017,11018,11019,11020,11021,11022,11023,11024,11025
7,-6.303448e+38,1.925032e+38,-8.382886e+38,4.803830e+39,-3.989533e+38,-8.090180e+38,1.419069e+40,-6.314012e+38,-7.860306e+38,-7.417590e+38,...,-8.610042e+38,4.608151e+39,4.471622e+39,-8.650836e+38,9.597904e+39,-3.020028e+38,6.265127e+39,-3.468848e+39,-4.958168e+38,-6.282597e+38
8,1.362395e+45,-5.698541e+44,1.838750e+45,-1.178439e+46,6.528313e+44,1.767523e+45,-2.181735e+46,1.325367e+45,1.722792e+45,1.666910e+45,...,1.946797e+45,-1.236126e+46,-1.141594e+46,1.909282e+45,-1.519295e+46,1.900969e+44,-1.315504e+46,6.244272e+45,9.966048e+44,1.161997e+45
9,-3.123210e+52,9.547564e+51,-4.153328e+52,2.380498e+53,-1.976404e+52,-4.008452e+52,7.030566e+53,-3.128325e+52,-3.894441e+52,-3.675255e+52,...,-4.266054e+52,2.283561e+53,2.213490e+53,-4.286134e+52,4.756941e+53,-1.497347e+52,3.104920e+53,-1.719044e+53,-2.456464e+52,-3.113095e+52


_Compare the performance of kernel SVMs with different kernels (polynomial kernels with different orders, Gaussian RBF with different bandwidths, etc.)._

I wrote a computegram_polynomial function to handle polynomial kernels with different orders, and would have been able write the code for a function to do the same for radial basis functions. However, since I can't get my fast gradient or straightforward (not fast) gradient descent algorithm to work with my implementations of the kernel SVM gradient and objective functions, I'm going to call it a day here.